In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i)
    def writeln(s): print('\n')
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [2]:
compileString("""

procedure eratosthenes() → (p: S)

program repeatprimes

""", 'test.wat', target = 'wat')

Exception: line 13 pos 30 illegal character

In [ ]:
# !wat2wasm test.wat

In [ ]:
# runwasm("test.wasm")